In [ ]:
import import_ipynb
from State_simulation_Final import State

import numpy as np
import matplotlib.pyplot as plt

from scipy.special import comb
from scipy.optimize import curve_fit
import scipy.integrate as integrate
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import norm

import qinfer
from qinfer import FiniteOutcomeModel
from qinfer import Distribution
from qinfer import SMCUpdater, UniformDistribution

from copy import deepcopy
import scipy.linalg as la

from qutip import basis, fock, tensor
from qutip import rx, ry, rz, sigmax, sigmay, sigmaz, rotation
from qutip import destroy, create, num, cnot, qeye
from qutip import expect, thermal_dm, ket2dm
from qutip import Qobj, QobjEvo, sesolve

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
interp_list = [[]]*4

# Each element of this list is an interpolator for 1, 3, 5 or 7 gates, respectively
# The interpolator is used to calculate the likelihood of a measurement (gg, ee, eg/ge) for
# a given Rabi detuning and a given number of gates

# For simplicitly, here we don't simulate with SPAM and depolarizing errors, but this 
# can easily be done as well (see LS_simulation_Final.ipynb)

psi=State([1,0,0,0,0])
d_range=np.linspace(0., 1.6, 51)

for j in range(4):

    probs = psi.detune(2*j+1, Sb=d_range, Rabi=d_range)
    interp = RegularGridInterpolator((d_range, d_range), probs, bounds_error=False, fill_value=0.)
    interp_list[j]=deepcopy(interp)

# 

# 

# 

In [ ]:
def plot(updater, i, mu, sig, cov, optimal_setting):

    
    plt.figure(figsize=(5, 12), dpi=150)
    
    ax = plt.subplot(411)
    plt.scatter(updater.particle_locations[:,0], updater.particle_locations[:,1], c=updater.particle_weights)
    plt.axis('equal')
    plt.xlabel('Sb')
    plt.ylabel('Rabi')

    
    ax = plt.subplot(412)
    steps=np.arange(i+1)
    ax.scatter(steps[:-1], 2*optimal_setting+1)
    plt.xlabel('step')
    plt.ylabel('# of gates')

    
    ax = plt.subplot(413)
    plt.plot(steps, mu)
    plt.legend(['$\mu_\Omega$', '$\mu_\delta$'])
    plt.grid()

    
    ax = plt.subplot(414)
    plt.plot(steps, sig)
    plt.plot(steps, np.array([np.linalg.det(cov[i]) for i in range(cov.shape[0])])**0.25)
    plt.legend(['$\sigma_\Omega$', '$\sigma_\delta$', '$\sqrt[4]{|\Sigma|}$'])
    plt.xscale('log')
    plt.yscale('log')
    plt.grid()

In [ ]:
class LSModel(FiniteOutcomeModel):
    
    @property
    def n_modelparams(self):
        # The model parameters (things that you want to optimize over, in this case, the drive power)
        return 2
    
    @property
    def is_n_outcomes_constant(self):
        return True
    def n_outcomes(self, expparams):
        return 3
    
    def are_models_valid(self, modelparams):
        
        return np.all(np.logical_and(modelparams > 0.2, modelparams < 1.4), axis=1)


    @property
    def expparams_dtype(self):
        return [('n_gates', 'int')]

    def likelihood(self, outcomes, modelparams, expparams):
        
        # Computes the likelihood of an outcome given a modelparam and an expparam (# of gates)
        
        super(LSModel, self).likelihood(
            outcomes, modelparams, expparams)
        
        expparams=np.squeeze(expparams)
        probabilities = interp_list[expparams]((modelparams[:, np.newaxis, 0], 
                                                  modelparams[:,np.newaxis, 1])).T.reshape(3, -1, 1).real
        # Computes the likelihood
        

        if len(np.shape(outcomes)) == 0:
            outcomes = np.array(outcomes)[None]
                    
        likelihood_array= np.concatenate([
            probabilities[np.newaxis, outcomes[idx]]
            for idx in range(outcomes.shape[0])
        ])
        # Generates the likelihood array for every outcome that you want to measure

        return likelihood_array

    
def particle_covariance_mtx(weights, locations):

        mu = np.dot(weights, locations)

        xs = locations.transpose([1, 0])

        ws = weights

        cov = (
            np.einsum('i,mi,ni', ws, xs, xs)
            - np.dot(mu[..., np.newaxis], mu[np.newaxis, ...])
        )

        assert np.all(np.isfinite(cov))
        if not np.all(la.eig(cov)[0] >= 0):
            warnings.warn('Numerical error in covariance estimation causing positive semidefinite violation.', ApproximationWarning)

        return cov
    
def expected_variance_decrease(updater, expparams):
        
        os = np.arange(3) # the possible outcomes, gg, ee, and eg/ge

        w_hyp, L, N = updater.hypothetical_update(
                os[:-1], 
                expparams, 
                return_normalization=True, 
                return_likelihood=True
            )
        
        # calculates the weights of the particles for each outcome
        
        w_hyp_last_outcome = (1 - L.sum(axis=0)) * updater.particle_weights[np.newaxis, :]
        N = np.concatenate([N[:,:,0], np.sum(w_hyp_last_outcome[np.newaxis,:,:], axis=2)], axis=0)
        w_hyp_last_outcome = w_hyp_last_outcome / N[-1,:,np.newaxis]
        w_hyp = np.concatenate([w_hyp, w_hyp_last_outcome[np.newaxis,:,:]], axis=0)

        
        cov = sum((np.linalg.det(particle_covariance_mtx(w_hyp[i,0], updater.particle_locations)))**0.25*N[i] for i in os)
        # calculates the expected covariance matrix after this measurement
        
        sig = (np.linalg.det(updater.est_covariance_mtx()))**0.25

        return sig-cov


def power2rabi(x, logpower):
    
    if logpower:
        
        # We don't now the precise relation between laser power and Rabi frequency, so here
        # we define some function that could be some power-to-Rabi relation. We choose to work here
        # with this log function because it seems to be qualitatively similar to the behavior of the power-to-Rabi
        # relation in the actual quantum computer.
        # The only requirement for this function is that at power=optimal power, Rabi=1
        
        return np.array([[np.log2(x[0]+1), x[1]]])
    else:
        return np.array([[x[0], x[1]]])
    
def measurement_loop(n_meas, n_shots_per_meas, model, updater, meas_settings, modelparams, 
                     thresh=None, plot_distributions=None, logpower=None):
    """
    
    This program takes an updater with a particle distribution that follows a uniform distribution over a
    specified range. At every step in range(n_meas) (or until it stops), it finds what is the best meas setting
    to use (how many gates to apply), simulates a measurement with that setting and with the "actual" rabi
    frequency, and with the outcome of that simulated measurement updates the position of the particles,
    reducing the variance of the distribution.
    
    
    :model: where we specify the outcomes, their likelihoods, their parameters, etc. (see LSModel above)
    :updater: the object that contains information about the particle locations and weights
    :meas_settings: whether we apply 1,3,5,7 gates, represented by an array [0,1,2,3]
    :modelparams: the "actual" values we want to obtain, for simulation purposes only, a list with a single value
    :n_meas, n_shots_per_meas: int's
    :thresh: float, the value of the particle distribution variance at which we stop the Bayesian update
    :plot_distributions: bool, in case you want to see the particle distribution and measurement outcomes at each step
    :logpower: bool, in case you want to use some power-to-Rabi relation that is not the trivial one (Rabi=power)
    
    
    """
    particle_locations=np.zeros((n_meas, 1000))
    particle_weights=np.zeros((n_meas, 1000))

    optimal_setting = np.zeros(n_meas)
    ydata = np.zeros((n_meas,n_shots_per_meas))
    mu = np.zeros((n_meas+1, model.n_modelparams))
    sig = np.zeros((n_meas+1, model.n_modelparams))
    
    mu[0] = updater.est_mean()
    sig[0] = variance(updater.particle_locations, updater.particle_weights)

    for i in tqdm(range(n_meas)):
        info_gain = [expected_variance_decrease(updater, np.array([meas_settings[i]])) 
                     for i in range(len(meas_settings))]

        opt_meas_set = np.array([meas_settings[np.argmax(info_gain)]]) 
        # Chooses the one that will minimize the variance the most
        
        datum = np.squeeze(model.simulate_experiment(power2rabi(modelparams, logpower),
                                                     opt_meas_set, repeat=n_shots_per_meas))
        # Simulates an experiment with the Rabi frequency given by the conversion, which the 
        # likelihood function doesn't "know"
        
        
        if plot_distributions:
            fig, ax = plt.subplots()
            ax.hist(updater.particle_locations[:,0]*mu[i,0]/mu[0,0], weights=updater.particle_weights)
            print(np.unique(datum, return_counts=True))
        
        for d in datum:
            updater.update(d, opt_meas_set)
            # updates the weights of the particles

        ydata[i] = datum
        optimal_setting[i] = opt_meas_set[0]

        mu[i+1] = mu[i]*updater.est_mean()
        
        sig[i+1] = variance(updater.particle_locations, updater.particle_weights)
        
        modelparams/=updater.est_mean()
        # updates the power of the next measurement, assuming that the relation between
        # Rabi and power is linear (because the optimizer doesn't "know" the power-to-Rabi relation)
        
        updater.particle_locations/=updater.est_mean()
        # updates the particle distribution, bringing the average to 0
        
        
        if thresh!=None:
            if np.all(np.sqrt(sig[i+1])<thresh):
                break # stops updating when variance reaches threshold
    
    mu/=mu[0]
    updater.particle_locations*=mu[i+1]

    ydata = 1-ydata.mean(axis=1)
    
    i+=1
    
    mu = mu[:i+1]
    sig = sig[:i+1]
    optimal_setting = optimal_setting[:i]
    ydata = ydata[:i+1]
    particle_locations=particle_locations[:i+1]
    particle_weights=particle_weights[:i+1]
    
    
    return updater, i, mu, sig, optimal_setting, ydata, modelparams

In [ ]:
model = LSModel()
modelparams = np.array([1.3, 0.8]) # The values at which the measurement outcomes will be generated
params_0=np.copy(modelparams)

prior = UniformDistribution([[.2, 1.4], [.2, 1.4]])
# We can start with a smaller range for the prior of the Rabi frequency, depending on 
# our current knowledge of possible laser powers.

# Notice that the range for the SPAM and depolarizing values here is 0. This is because
# we don't want the optimizer to learn them, as in the real experiment. We can see that
# the optimization works even if the SPAM and depolarizing values that the optimizer assumes
# is different from the values at which the measurement data is generated

updater = SMCUpdater(model, 1000, prior)
# Creates an updater with 1000 particles

meas_settings = np.arange(4).astype(int) # 0, 1, 2, 3 corresponds to 1, 3, 5, 7 gates respectively

n_meas = 10

n_shots_per_meas = 100

thresh=0.01

updater, i, mu, sig, optimal_setting, ydata, params = measurement_loop(
    n_meas, n_shots_per_meas, model, deepcopy(updater),
    meas_settings, modelparams, thresh, plot_distributions=False)

plot(updater, i, mu, np.sqrt(sig), optimal_setting, params_0)

print("Finished in {} steps with values Sb={:.4f}$\pm${:.4f}, Rabi={:.4f}$\pm${:.4f}".format(
    i-1, mu[-1,0], sig[-1,0], mu[-1,1], sig[-1,1]))